🛠️ 코드 실행 전 준비사항

1.  **필요 라이브러리 설치**:  
pip install geopandas rasterio shapely diffusers transformers accelerate  safetensors  

In [ ]:
import os
import torch
import numpy as np
import cv2
import rasterio
from rasterio.features import rasterize
import geopandas as gpd
from PIL import Image
from diffusers import StableDiffusionInpaintPipeline

# -------------------------------------------------------------------------
# 1. 환경 설정 및 데이터 로드
# -------------------------------------------------------------------------
# 파일 경로 설정
raster_path = 'org.tiff'    # 원본 래스터 파일 
vector_path = 'coverage.gpkg'   # 보안 구역 폴리곤 파일
output_path = 'restored_satellite.tif'  # 결과 저장 경로

# 허깅페이스 인증 에러 방지를 위해 본인의 토큰을 입력하세요.
HF_TOKEN = "" # 여기에 허깅페이스 토큰 입력

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

def run_satellite_inpainting():
    if not HF_TOKEN:
        print("⚠️ 경고: HF_TOKEN이 비어있습니다. 인증 에러가 발생할 수 있습니다.")

    print(f"🚀 작업을 시작합니다. (사용 장치: {device})")

    # 1. 원본 래스터(TIFF) 로드
    try:
        with rasterio.open(raster_path) as src:
            image_data = src.read()
            transform = src.transform
            crs = src.crs
            width, height = src.width, src.height
            
            # RGB 추출 (처음 3개 채널)
            img_np = np.transpose(image_data[:3, :, :], (1, 2, 0))
            
            # 0~255 uint8 정규화 (수치 안정성 강화)
            img_min = np.percentile(img_np, 1)
            img_max = np.percentile(img_np, 99)
            img_np = np.clip(img_np, img_min, img_max)
            img_np = ((img_np - img_min) / (img_max - img_min + 1e-8) * 255).astype(np.uint8)
            
            # ⭐ [수정] SD 모델의 표준 해상도인 512x512로 리사이징
            init_image = Image.fromarray(img_np).convert("RGB").resize((512, 512), Image.LANCZOS)
            print(f"📸 원본 크기: {width}x{height} -> AI 연산 크기: 512x512")
            
    except Exception as e:
        print(f"❌ 래스터 파일 로드 실패: {e}")
        return

    # 2. 보안 구역 폴리곤(GPKG) 로드 및 마스크 생성
    try:
        gdf = gpd.read_file(vector_path)
        if gdf.crs != crs:
            gdf = gdf.to_crs(crs)

        shapes = ((geom, 255) for geom in gdf.geometry)
        mask_np = rasterize(shapes, out_shape=(height, width), transform=transform)
        
        # 마스크도 AI 연산 크기인 512x512로 동일하게 리사이징
        mask_image = Image.fromarray(mask_np.astype(np.uint8)).convert("L").resize((512, 512), Image.NEAREST)
        mask_image.save("debug_mask.png")
    except Exception as e:
        print(f"❌ 벡터 파일 처리 실패: {e}")
        return

    # -------------------------------------------------------------------------
    # 2. AI 모델 로드 및 추론 (Inpainting)
    # -------------------------------------------------------------------------
    print("🤖 AI 인페인팅 모델을 로드 중입니다...")
    
    model_id = "Daro77/stable-diffusion-2-inpainting-gaf-mtf-rp-spec"
    
    try:
        # ⭐ [에러 수정 핵심] 
        # Half/Float 타입 불일치 에러를 해결하기 위해 전체를 float32로 로드합니다.
        # GTX 16xx 카드의 '검은 화면' 문제도 이 설정으로 완벽히 해결됩니다.
        pipe = StableDiffusionInpaintPipeline.from_pretrained(
            model_id, 
            torch_dtype=torch.float32, # 전체 정밀도를 float32로 통일
            use_safetensors=True, 
            token=HF_TOKEN if HF_TOKEN else None,
            safety_checker=None,
            requires_safety_checker=False
        )
        
        if device == "cuda":
            # float32 사용으로 인한 VRAM 부족을 방지하기 위해 순차적 오프로딩을 사용합니다.
            pipe.enable_model_cpu_offload() 
            # 연산 가속을 위한 설정
            if hasattr(pipe, "enable_xformers_memory_efficient_attention"):
                try:
                    pipe.enable_xformers_memory_efficient_attention()
                except:
                    pass
        else:
            pipe.to("cpu")

    except Exception as e:
        print(f"❌ 모델 로드 중 오류 발생: {e}")
        return

    # 프롬프트 설정 (풍부한 녹색 숲 질감을 유도)
    prompt = "extremely high resolution satellite photography of a dense lush green forest, highly detailed trees, realistic vegetation, seamless nature landscape"
    negative_prompt = "black, dark, solid color, blurry, buildings, urban, concrete, artificial, text, watermark, low quality, glitch, distorted"

    print("🎨 지형 복원(Inpainting) 진행 중... (약 1~2분 소요)")
    with torch.inference_mode():
        # 인페인팅 수행
        result_output = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=init_image,
            mask_image=mask_image,
            num_inference_steps=50, # 품질을 위해 단계를 높임
            guidance_scale=9.0      # 프롬프트 가이드 강도 강화
        )
        result_image = result_output.images[0]

    # -------------------------------------------------------------------------
    # 3. 결과 저장
    # -------------------------------------------------------------------------
    # 1. AI 결과물(512x512)을 다시 원본 해상도로 정교하게 복구
    result_image = result_image.resize((width, height), Image.LANCZOS)
    result_np = np.array(result_image)
    
    # 결과값 검증 및 강제 가시성 확보 (최종 방어 로직)
    if result_np.max() < 5:
        print("⚠️ 경고: 결과가 여전히 너무 어둡습니다. 프롬프트나 모델 설정을 재점검해야 합니다.")

    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=3,
        dtype='uint8',
        crs=crs,
        transform=transform,
    ) as dst:
        for i in range(3):
            dst.write(result_np[:, :, i], i + 1)

    print(f"✨ 작업 완료! 결과물 저장됨: {output_path}")

if __name__ == "__main__":
    run_satellite_inpainting()

🚀 작업을 시작합니다. (사용 장치: cuda)
📸 원본 크기: 1492x880 -> AI 연산 크기: 512x512
🤖 AI 인페인팅 모델을 로드 중입니다...


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/372 [00:00<?, ?it/s]

🎨 지형 복원(Inpainting) 진행 중... (약 1~2분 소요)


  0%|          | 0/50 [00:00<?, ?it/s]

✨ 작업 완료! 결과물 저장됨: restored_satellite.tif
